In [5]:
# use open cv to show new images from AirSim 

import setup_path 
import airsim

# requires Python 3.5.3 :: Anaconda 4.4.0

# pip install opencv-python
import cv2
import time
import math
import sys
import numpy as np
import time


In [6]:
from PIL import Image

def getScreenDepthVis():

    responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPerspective, True, False)])
    img1d = np.array(responses[0].image_data_float, dtype=np.float)
    img1d = 255/np.maximum(np.ones(img1d.size), img1d)
    img2d = np.reshape(img1d, (responses[0].height, responses[0].width))

    image = np.invert(np.array(Image.fromarray(img2d.astype(np.uint8), mode='L')))

    factor = 10
    maxIntensity = 255.0 # depends on dtype of image data

    # Decrease intensity such that dark pixels become much darker, bright pixels become slightly dark 
    newImage1 = (maxIntensity)*(image/maxIntensity)**factor
    newImage1 = np.array(newImage1,dtype=np.uint8)

    #cv2.imshow("Test", newImage1)
    #cv2.waitKey(0)

    return newImage1

In [7]:
import matplotlib.pyplot as plt

In [8]:
client = airsim.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
client.takeoffAsync().join()
# client.moveByVelocityAsync(0,0,-5,10)
# client.hoverAsync()
# you must first press "1" in the AirSim view to turn on the depth capture

# get depth image
yaw = 0
pi = 3.14159265483
vx = 0
vy = 0
driving = 0
help = False

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [9]:
client.moveToPositionAsync(0,0, -1.5,6, )

In [10]:
# client.landAsync().join()

In [11]:
from PIL import Image

responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPerspective, True, False)])
img1d = np.array(responses[0].image_data_float, dtype=np.float)
img1d = 255/np.maximum(np.ones(img1d.size), img1d)
img2d = np.reshape(img1d, (responses[0].height, responses[0].width))
        
image = np.invert(np.array(Image.fromarray(img2d.astype(np.uint8), mode='L')))
        
factor = 10
maxIntensity = 255.0 # depends on dtype of image data
        
        # Decrease intensity such that dark pixels become much darker, bright pixels become slightly dark 
newImage1 = (maxIntensity)*(image/maxIntensity)**factor
newImage1 = np.array(newImage1,dtype=np.uint8)
        
        #cv2.imshow("Test", newImage1)
        #cv2.waitKey(0)
        
#         return newImage1

In [12]:
destination = [-120,190]
fly_speed=5
turn_speed=0.5
thrust_speed = 0.2
while True:
    # this will return png width= 256, height= 144
    
    result = client.simGetImage("0", airsim.ImageType.DepthVis)
    if (result == "\0"):
        if (not help):
            help = True
            print("Please press '1' in the AirSim view to enable the Depth camera view")
    else:    
#         rawImage = np.fromstring(result, np.int8)
#         print("rawImage is",rawImage)
#         png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
#         gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)
        gray = getScreenDepthVis()
        # slice the image so we only check what we are headed into (and not what is down on the ground below us).
        top = np.vstack((np.vsplit(gray, 3)[0],np.vsplit(gray, 3)[1]))
        #top = np.vsplit(gray, 3)[0]

        # now look at 4 horizontal bands (far left, left, right, far right) and see which is most open.
        # the depth map uses black for close (0) and white for far away(255), so we invert that
        # to get an estimate of distance.
        bands = np.hsplit(top, [50,100,150,200]);
        mins = [np.min(x) for x in bands]
        max = np.argmax(mins)    
        distance = mins[max]

        # sanity check on what is directly in front of us (slot 2 in our hsplit)
        current = mins[2]
        print ("max distance=", current)
        print ("distance=", current)
        print("current position is",client.getMultirotorState().kinematics_estimated.position)
#         if (current <20):
#             client.hoverAsync().join()
#             airsim.wait_key("whoops - we are about to crash, so stopping!")
        
        pitch, roll, yaw  = airsim.to_eularian_angles(client.simGetVehiclePose().orientation)
        
        if (distance > current + 10) and current<150:
        
            # we have a 90 degree field of view (pi/2), we've sliced that into 5 chunks, each chunk then represents
            # an angular delta of the following pi/10.
            change = 0
            driving = max
            if (max == 0):
                change = -2 * pi / 10
            elif (max == 1):
                change = -pi / 10
            elif (max == 2):
                change = 0 # center strip, go straight
            elif (max == 3):
                change = pi / 10
            else:
                change = 2*pi/10
    
            yaw = (yaw + change)
            vx = turn_speed*math.cos(yaw);
            vy = turn_speed*math.sin(yaw);
            print ("switching angle", math.degrees(yaw), vx, vy, min, distance, current)
            #client.getMultirotorState().kinematics_estimated.position
            #print("current position is",client.getMultirotorState().kinematics_estimated.position)
            if (vx == 0 and vy == 0):
                vx = math.cos(yaw);
                vy = math.sin(yaw);
            #print ("distance=", current)
            client.moveByVelocityZAsync(fly_speed*vx, fly_speed*vy,-12, 1, airsim.DrivetrainType.ForwardOnly, airsim.YawMode(False, 0)).join()
            
        else:
            curr_position = client.getMultirotorState().kinematics_estimated.position
            distance_x = destination[0]-curr_position.x_val
            distance_y = destination[1]-curr_position.y_val
            norm = np.linalg.norm([distance_x,distance_y])
            if (norm<5):
                print('Trigger landing')
                client.moveByVelocityZAsync(0, 0,-12, 1, airsim.DrivetrainType.ForwardOnly, airsim.YawMode(False, 0)).join()
                break
            vx = thrust_speed*distance_x/norm
            vy = thrust_speed*distance_y/norm
            client.moveByVelocityZAsync(fly_speed*vx, fly_speed*vy,-12, 1, airsim.DrivetrainType.ForwardOnly, airsim.YawMode(False, 0)).join()

            
    


        x = int(driving * 50)
        cv2.rectangle(gray, (x,0), (x+50,50), (0,255,0), 2)
        cv2.imshow("Top", top)

    key = cv2.waitKey(1) & 0xFF;
    if (key == 27 or key == ord('q') or key == ord('x')):
        break;
print('out of loop')
time.sleep(5)
client.hoverAsync()
time.sleep(5)
curr_position = client.getMultirotorState().kinematics_estimated.position
distance_x = destination[0]-curr_position.x_val
distance_y = destination[1]-curr_position.y_val
# norm = np.linalg.norm([distance_x,distance_y])
client.moveToPositionAsync(curr_position.x_val, curr_position.y_val, -1.5,3, )
time.sleep(10)
client.landAsync().join()
# # client.moveToPositionAsync(destination[0], destination[1], -25,3, )      
# client.moveToPositionAsync(curr_position.x_val, curr_position.y_val, -1.5,3, )
# client.landAsync().join()

max distance= 107
distance= 107
current position is <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': -2.1444413661956787}
max distance= 58
distance= 58
current position is <Vector3r> {   'x_val': -0.2894294261932373,
    'y_val': 0.45883771777153015,
    'z_val': -5.566182613372803}
switching angle 176.80742066700245 -0.49922399276709667 0.02784609569899929 <built-in function min> 127 58
max distance= 127
distance= 127
current position is <Vector3r> {   'x_val': -1.4273583889007568,
    'y_val': 0.9999563097953796,
    'z_val': -10.242979049682617}
switching angle -161.84482485780018 -0.4751080566521091 -0.15579581029108672 <built-in function min> 178 127
max distance= 151
distance= 151
current position is <Vector3r> {   'x_val': -3.4251673221588135,
    'y_val': 1.1861504316329956,
    'z_val': -11.7735013961792}
max distance= 8
distance= 8
current position is <Vector3r> {   'x_val': -5.357806205749512,
    'y_val': 1.3304822444915771,
    'z_val': -11.996803283691406}
swit

max distance= 139
distance= 139
current position is <Vector3r> {   'x_val': -15.16600227355957,
    'y_val': 63.563629150390625,
    'z_val': -11.968937873840332}
max distance= 90
distance= 90
current position is <Vector3r> {   'x_val': -16.613929748535156,
    'y_val': 64.52719116210938,
    'z_val': -12.013311386108398}
switching angle 166.35415985057244 -0.4858862806814059 0.11795983318736118 <built-in function min> 157 90
max distance= 117
distance= 117
current position is <Vector3r> {   'x_val': -18.248987197875977,
    'y_val': 65.42777252197266,
    'z_val': -11.967127799987793}
max distance= 48
distance= 48
current position is <Vector3r> {   'x_val': -20.00193977355957,
    'y_val': 66.29948425292969,
    'z_val': -12.022808074951172}
switching angle 165.90834047468172 -0.4849537338700392 0.12173691307695948 <built-in function min> 139 48
max distance= 72
distance= 72
current position is <Vector3r> {   'x_val': -21.877256393432617,
    'y_val': 67.18684387207031,
    'z_val': -

max distance= 185
distance= 185
current position is <Vector3r> {   'x_val': -64.99747467041016,
    'y_val': 104.70298767089844,
    'z_val': -11.997652053833008}
max distance= 178
distance= 178
current position is <Vector3r> {   'x_val': -65.6367416381836,
    'y_val': 105.69537353515625,
    'z_val': -11.997652053833008}
max distance= 178
distance= 178
current position is <Vector3r> {   'x_val': -66.27921295166016,
    'y_val': 106.69273376464844,
    'z_val': -11.997652053833008}
max distance= 170
distance= 170
current position is <Vector3r> {   'x_val': -66.90245819091797,
    'y_val': 107.66024780273438,
    'z_val': -11.997652053833008}
max distance= 170
distance= 170
current position is <Vector3r> {   'x_val': -67.5417251586914,
    'y_val': 108.65263366699219,
    'z_val': -11.997652053833008}
max distance= 164
distance= 164
current position is <Vector3r> {   'x_val': -68.17137908935547,
    'y_val': 109.63009643554688,
    'z_val': -11.997652053833008}
max distance= 164
distan

max distance= 50
distance= 50
current position is <Vector3r> {   'x_val': -107.85640716552734,
    'y_val': 126.41075897216797,
    'z_val': -12.005574226379395}
switching angle 167.77888619051294 -0.48866897678739407 0.10584248261242415 <built-in function min> 117 50
max distance= 66
distance= 66
current position is <Vector3r> {   'x_val': -108.92424011230469,
    'y_val': 127.44722747802734,
    'z_val': -11.976204872131348}
switching angle 186.13542396525276 -0.49713602737804946 -0.053439407582525106 <built-in function min> 86 66
max distance= 164
distance= 164
current position is <Vector3r> {   'x_val': -110.65985107421875,
    'y_val': 128.075439453125,
    'z_val': -11.972256660461426}
max distance= 9
distance= 9
current position is <Vector3r> {   'x_val': -112.34443664550781,
    'y_val': 128.62425231933594,
    'z_val': -12.029203414916992}
switching angle 50.06729754518093 0.3209436992355023 0.3833994547740404 <built-in function min> 20 9
max distance= 5
distance= 5
current po

max distance= 44
distance= 44
current position is <Vector3r> {   'x_val': -122.44970703125,
    'y_val': 184.61260986328125,
    'z_val': -12.02815055847168}
switching angle 109.17094989489733 -0.16419389339900997 0.47227149540330565 <built-in function min> 209 44
max distance= 200
distance= 200
current position is <Vector3r> {   'x_val': -122.74459838867188,
    'y_val': 186.64541625976562,
    'z_val': -11.971839904785156}
Trigger landing
out of loop


# End Here

In [ ]:
client.hoverAsync()

In [ ]:
curr_position = client.getMultirotorState().kinematics_estimated.position
distance_x = destination[0]-curr_position.x_val
distance_y = destination[1]-curr_position.y_val
# norm = np.linalg.norm([distance_x,distance_y])
client.moveToPositionAsync(curr_position.x_val, curr_position.y_val, -1.5,6, )


In [ ]:
client.landAsync().join()

In [16]:
client.moveToPositionAsync(curr_position.x_val, curr_position.y_val, -1.5,6, )


RuntimeError: IOLoop is already running

In [17]:
client.landAsync().join()
client.landAsync().join()

RuntimeError: IOLoop is already running

In [9]:
curr_position = client.getMultirotorState().kinematics_estimated.position

In [19]:
client.moveToPositionAsync(curr_position.x_val, curr_position.y_val, -1.5,3, )
client.landAsync().join()

RuntimeError: IOLoop is already running

In [16]:
client.moveToPositionAsync(destination[0], destination[1], -25,3, )      
client.moveToPositionAsync(curr_position.x_val, curr_position.y_val, -1.5,3, )
client.landAsync().join()

KeyboardInterrupt: 

In [26]:
client.moveToPositionAsync(curr_position.x_val, curr_position.y_val, -1.5,
        3, )
client.landAsync().join()

In [19]:
client.moveByVelocityZAsync(0,0,-5, 1, airsim.DrivetrainType.ForwardOnly, 
                            airsim.YawMode(False, 0)).join()

In [ ]:
(fly_speed*vx, fly_speed*vy,-25, 1, airsim.DrivetrainType.ForwardOnly, airsim.YawMode(False, 0)).join()

In [ ]:
plt.imshow(newImage1,plt.cm.gray)

In [ ]:
response=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPlanner, pixels_as_float=True)])[0]

In [ ]:
depth = np.array(response.image_data_float)
depth = depth.reshape(response.height, response.width)
#depth = np.array(depth * 255, dtype=np.uint8)


In [ ]:
 np.array(response.image_data_float)[160:180]

In [ ]:
result = client.simGetImage("0", airsim.ImageType.Segmentation)

rawImage = np.fromstring(result, np.int8)
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(png)

In [ ]:
 np.array(depth * 255, dtype=np.uint8)

In [ ]:
depth = np.array(response.image_data_float)
depth = depth.reshape(response.height, response.width)

In [ ]:
depth [0][160:180]

In [ ]:
plt.imshow(depth,plt.cm.gray)

In [ ]:
img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8)
#img_rgb = img1d.reshape(response.height, response.width, 3) # reshape array to 3 channel image array H X W X 3
cv2.imwrite("./try.png", img1d) # write to png 

In [ ]:
img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) # get numpy array
img_rgb = img1d.reshape(response.height, response.width, 3) # reshape array to 3 channel image array H X W X 3

In [ ]:
len(img1d)

In [ ]:
result_scene =client.simGetImage("0", airsim.ImageType.Scene)
rawImage = np.fromstring(result_scene, np.int8)

In [ ]:
len(rawImage)

In [ ]:
response=(client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthVis)])[0])

In [ ]:
tmp=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthVis,False, False)])[0].image_data_uint8

In [ ]:
tmp=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthVis, pixels_as_float=True)])[0]

In [ ]:
rawImage = np.fromstring(tmp, np.int8)

In [ ]:
rawImage = np.frombuffer(tmp, np.int8)

In [ ]:
tmp

In [ ]:
plt.imshow(gray)

In [ ]:
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
color = cv2.cvtColor(depth, cv2.COLOR_GRAY2RNG)

In [ ]:
depth[0]

In [ ]:
depth = np.array(response.image_data_float)
depth = depth.reshape(response.height, response.width)
#depth = np.array(depth * 253, dtype=np.uint8)

In [ ]:
depth

In [ ]:
(depth * 255)[0][160:180]

In [ ]:
plt.imshow(depth,)

In [ ]:
response.image_data_float[160:180]

In [ ]:
depth[0]

In [ ]:
depth[0][160:180]

In [ ]:
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)

In [ ]:
response.image_data_uint8

In [ ]:
plt.imshow(depth,plt.cm.gray)

In [ ]:
result_scene =client.simGetImage("0", airsim.ImageType.Scene)
rawImage = np.fromstring(result_scene, np.int8)
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)

In [ ]:
result =client.simGetImage("4", airsim.ImageType.DepthPlanner)
rawImage = np.fromstring(result, np.int8)
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
rawImage=np.int8(np.ones(np.size(rawImage))*0)+rawImage

In [ ]:
np.unique(png)

In [ ]:
rawImage=np.ones(np.size(rawImage))*0+rawImage
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
##gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
response=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPlanner,False, False)])[0]
img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) # get numpy array
img_rgb = img1d.reshape(response.height, response.width, 3) # reshape array to 4 channel image array H X W X 3
cv2.imwrite(os.path.normpath(filename + '.png'), img_rgb) # write to png

In [ ]:
cv2.imwrite("./try2.png",response.image_data_uint8)

In [ ]:
fh = open("./try2.png", "wb")
fh.write(response.image_data_uint8)
fh.close()

In [ ]:
png = cv2.imdecode(img1d, cv2.IMREAD_UNCHANGED)

In [ ]:
np.unique(png)

In [ ]:
# call airsim
responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.Scene, True)])
response = responses[0]
# convert to grayscale
depth = np.array(response.image_data_float, dtype=np.float32)
depth = depth.reshape(response.height, response.width)
depth = np.array(depth * 255, dtype=np.uint8)
# save pic
# cv2.imwrite('./depth.png'.format(name_folder, t), depth)

In [ ]:
cv2.imwrite("./filename.png", depth) 

In [ ]:
rawImage = np.fromstring(tmp, np.int8)

In [ ]:
result_depth = client.simGetImage("0", airsim.ImageType.DepthVis)
rawImage = np.fromstring(result_depth, np.int8)

In [ ]:
np.size(rawImage)

In [ ]:
result_DepthPlanner = client.simGetImage("0", airsim.ImageType.DepthPlanner)

In [ ]:
# result_DepthPlanner = client.simGetImage("0", airsim.ImageType.DepthPlanner)
result_DepthPlanner=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPlanner, pixels_as_float=True)])

In [ ]:
rawImage = np.fromstring(result_DepthPlanner, np.int8)

In [ ]:
result_DisparityNormalized = client.simGetImage("0", airsim.ImageType.DisparityNormalized)
rawImage = np.fromstring(result_DisparityNormalized, np.int8)

In [ ]:
result_Segmentation  = client.simGetImage("0", airsim.ImageType.Segmentation)
rawImage = np.fromstring(result_Segmentation, np.int8)

In [ ]:
rawImage = np.fromstring(result_DepthPlanner, np.int8)

In [ ]:
png = cv2.imdecode(rawImage, cv2.IMREAD_GRAYSCALE)
# gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(png)

In [ ]:
# call airsim
responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthVis, True)])
response = responses[0]
# convert to grayscale
depth = np.array(response.image_data_float, dtype=np.float32)
depth = depth.reshape(response.height, response.width)
depth = np.array(depth * 255, dtype=np.uint8)
# save pic
# cv2.imwrite('{}/depth{}.png'.format(self.name_folder, self.t), depth)

In [ ]:
plt.imshow(depth)

In [ ]:
np.size(rawImage)

In [ ]:
np.reshape(rawImage,[2,2389])

In [ ]:
len(rawImage)/2

In [ ]:
plt.imshow(png)

In [ ]:
import numpy as np
import urllib
import cv2

url = 'http://www.pyimagesearch.com/wp-content/uploads/2015/01/google_logo.png'
resp = urllib.urlopen(url)
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)
cv2.imshow('URL2Image',image)

In [ ]:
plt.imshow(png)

In [ ]:
top = np.vsplit(gray, 3)[0]

In [ ]:
np.vsplit(gray, 2)

In [ ]:
bands = np.hsplit(top, [50,100,150,200]);

In [ ]:
        maxes = [np.max(x) for x in bands]
        min = np.argmin(maxes)    
        distance = 255 - maxes[min]

        # sanity check on what is directly in front of us (slot 2 in our hsplit)
        current = 255 - maxes[2]

In [ ]:
plt.imshow(bands[5])

In [ ]:
gray = getScreenDepthVis()
# slice the image so we only check what we are headed into (and not what is down on the ground below us).

top = np.vsplit(gray, 2)[0]

# now look at 4 horizontal bands (far left, left, right, far right) and see which is most open.
# the depth map uses black for far away (0) and white for very close (255), so we invert that
# to get an estimate of distance.
bands = np.hsplit(top, [50,100,150,200]);
maxes = [np.max(x) for x in bands]
min = np.argmin(maxes)    
distance = 255 - maxes[min]

# sanity check on what is directly in front of us (slot 2 in our hsplit)
current = 255 - maxes[2]

In [ ]:
bands

In [ ]:
plt.imshow(gray,plt.cm.gray)

In [5]:
fly_speed=10
turn_speed=0.5
while True:
    # this will return png width= 256, height= 144
    result = client.simGetImage("0", airsim.ImageType.DepthVis)
    if (result == "\0"):
        if (not help):
            help = True
            print("Please press '1' in the AirSim view to enable the Depth camera view")
    else:    
#         rawImage = np.fromstring(result, np.int8)
#         print("rawImage is",rawImage)
#         png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
#         gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)
        gray = getScreenDepthVis()
        # slice the image so we only check what we are headed into (and not what is down on the ground below us).
        top = np.vstack((np.vsplit(gray, 3)[0],np.vsplit(gray, 3)[1]))
        #top = np.vsplit(gray, 3)[0]

        # now look at 4 horizontal bands (far left, left, right, far right) and see which is most open.
        # the depth map uses black for close (0) and white for far away(255), so we invert that
        # to get an estimate of distance.
        bands = np.hsplit(top, [50,100,150,200]);
        mins = [np.min(x) for x in bands]
        max = np.argmax(mins)    
        distance = mins[max]

        # sanity check on what is directly in front of us (slot 2 in our hsplit)
        current = mins[2]
        print ("max distance=", current)
        print ("distance=", current)
#         if (current <20):
#             client.hoverAsync().join()
#             airsim.wait_key("whoops - we are about to crash, so stopping!")
    
        pitch, roll, yaw  = airsim.to_eularian_angles(client.simGetVehiclePose().orientation)

        if (distance > current + 10):
        
            # we have a 90 degree field of view (pi/2), we've sliced that into 5 chunks, each chunk then represents
            # an angular delta of the following pi/10.
            change = 0
            driving = max
            if (max == 0):
                change = -2 * pi / 10
            elif (max == 1):
                change = -pi / 10
            elif (max == 2):
                change = 0 # center strip, go straight
            elif (max == 3):
                change = pi / 10
            else:
                change = 2*pi/10
    
            yaw = (yaw + change)
            vx = turn_speed*math.cos(yaw);
            vy = turn_speed*math.sin(yaw);
            print ("switching angle", math.degrees(yaw), vx, vy, min, distance, current)
    
        if (vx == 0 and vy == 0):
            vx = math.cos(yaw);
            vy = math.sin(yaw);

        #print ("distance=", current)
        client.moveByVelocityZAsync(fly_speed*vx, fly_speed*vy,-25, 1, airsim.DrivetrainType.ForwardOnly, airsim.YawMode(False, 0)).join()

        x = int(driving * 50)
        cv2.rectangle(gray, (x,0), (x+50,50), (0,255,0), 2)
        cv2.imshow("Top", top)

    key = cv2.waitKey(1) & 0xFF;
    if (key == 27 or key == ord('q') or key == ord('x')):
        break;
        


max distance= 107
distance= 107
switching angle 215.99995903307672 -0.40450870732285976 -0.2938923369194729 <built-in function min> 127 107
max distance= 107
distance= 107
max distance= 185
distance= 185
switching angle -107.5013546854304 -0.15036417441890074 -0.47685492033879906 <built-in function min> 209 185
max distance= 200
distance= 200
max distance= 193
distance= 193
max distance= 193
distance= 193
max distance= 185
distance= 185
max distance= 178
distance= 178
max distance= 178
distance= 178
max distance= 164
distance= 164
max distance= 144
distance= 144
switching angle -71.5011122267247 0.1586431237345445 -0.47416490727567134 <built-in function min> 170 144
max distance= 151
distance= 151
switching angle -91.07138983485274 -0.009349095234157 -0.49991258677723116 <built-in function min> 170 151
max distance= 164
distance= 164
max distance= 151
distance= 151
max distance= 127
distance= 127
max distance= 144
distance= 144
switching angle -109.07081585041158 -0.16336826892344977 -

KeyboardInterrupt: 

In [ ]:
bands

In [ ]:
rawImage = np.fromstring(result, np.int8)